### Load DIC data

In [168]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from DICEBSD.quat import Quat
import DICEBSD.ebsd as ebsd
import DICEBSD.hrdic as hrdic
from IPython.display import display
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.filters import threshold_local, sobel
import skimage.morphology as mph
from skimage import feature, measure
import scipy.ndimage as nd
from skimage.transform import rescale, resize, downscale_local_mean
from scipy import ndimage
import matplotlib as mpl

%matplotlib osx

#Load in DIC maps

DicFilePath = "/Users/mbcx9rt5/Dropbox (Research Group)/new davis/zrh/"
SetA1 = hrdic.Map(DicFilePath, "SetA_001.TXT")
SetA1.setPatternPath("SetA_opt.jpg", 4)
SetA.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

SetA2 = hrdic.Map(DicFilePath, "SetA_002.TXT")
SetA2.setPatternPath("SetA_opt.jpg", 4)
SetA2.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

SetA3 = hrdic.Map(DicFilePath, "SetA_003.TXT")
SetA3.setPatternPath("SetA_opt.jpg", 4)
SetA3.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

SetC = hrdic.Map(DicFilePath, "SetC_002.TXT")
SetC.setPatternPath("SetC_opt.jpg", 4)
SetC.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

SetD = hrdic.Map(DicFilePath, "SetD_002.TXT")
SetD.setPatternPath("SetD_opt.jpg", 4)
SetD.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)


### Select DIC map

In [287]:
DicMap=SetA3

print("Max Exx: {0:.2f}%".format(np.max(DicMap.crop(DicMap.f11))*100))
print("Max shear: {0:.2f}%".format(np.max(DicMap.crop(DicMap.max_shear))*100))
print("Min shear: {0:.2f}%".format(np.min(DicMap.crop(DicMap.max_shear))*100))
print("Mean Exx: {0:.2f}%".format(np.mean(DicMap.crop(DicMap.f11))*100))
print("Mean shear: {0:.2f}%".format(np.mean(DicMap.crop(DicMap.max_shear))*100))
print("Points: {0}".format(len(DicMap.crop(DicMap.max_shear)[0])*len(DicMap.crop(DicMap.max_shear)[1])))

Max Exx: 96.02%
Max shear: 85.38%
Min shear: 0.00%
Mean Exx: 3.01%
Mean shear: 2.65%
Points: 1510441


### Load and threshold BSE image

In [170]:
image = plt.imread(DicMap.patternImPath)
image = nd.filters.median_filter(image,size=16)

In [267]:
thres = threshold_local(image, 55, offset=10)

binary = image < thresh - 20
binary = rescale(binary, 1.0 / 4.0)
binary = mph.binary_closing(binary)
binary=mph.remove_small_objects(binary,min_size=200,connectivity=1)

binary2 = image < 30
binary2 = rescale(binary, 1.0 / 4.0)

/Users/mbcx9rt5/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/mbcx9rt5/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [268]:
plt.figure(figsize=(16,10))
plt.imshow(binary)
plt.show()

In [269]:
edges = sobel(binary)
edges[edges>=0.01] = 1
edges[edges<0.01] = 0

In [270]:
plt.figure(figsize=(16,10))
plt.imshow(edges)
plt.show()

### Plot DIC and skeleton

In [274]:
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1,cmap1.N + 3)

In [290]:
fig = plt.figure(frameon=False,figsize=(8,7))
ax = plt.Axes(fig, [0., 0., 0.9, 1.])
#fig,ax=plt.subplots(1, 1, figsize=(10, 8),frameon=False)
ax.set_axis_off()
fig.add_axes(ax)

img = ax.imshow(DicMap.crop(DicMap.max_shear)*100,vmax=12)
ax.imshow(DicMap.crop(edges),cmap=cmap1)

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

scalebar = ScaleBar(0.078125e-6) # 1 pixel = 0.2 meter
plt.gca().add_artist(scalebar)

cbar=plt.colorbar(img, ax=ax,fraction=0.03, pad=0.02, anchor=(0.0, 0.22))
cbar.set_label(label="Effective shear strain (%)",size=12)
cbar.ax.tick_params(labelsize=11) 
plt.show()

#fig.savefig('fig2.png',dpi=600, bbox_inches='tight', pad_inches=0)

##### 